# Use AI classifier to spot events

We will see how a trained AI on simulation data is able to spot a dyfunction, here namely a broken fan, in operation data.

### Load the AI model

In [ ]:
import numpy as np
import pandas as pd
import pickle

# load the already trained classifier and
classifier = pickle.load(open("data/broken_fan_classifier.pkl", "rb"))
classifier_mean = pd.read_csv("data/broken_fan_classifier_mean.csv", index_col=0)
classifier_std = pd.read_csv("data/broken_fan_classifier_std.csv", index_col=0)

### Load the operation data

In [ ]:
# load the operation data to check for borken fan event
data = pd.read_csv("data/cpu_hot_day_intensive_use_broken.csv", index_col=0)
# do data normalization
classifier_data = (
    data[["fan.T_air", "T_cpu", "fan.tension"]] - classifier_mean.values.T
) / classifier_std.values.T
classifier_data.T

### Perform the checks

In [ ]:
# use the AI classifier to predict nominal vs broken fan
diag = pd.DataFrame(
    np.stack((classifier_data.index, classifier.predict(classifier_data.values)), axis=1),
    columns=["time", "fan_working_diagnostic"],
)

# also get associated statistics
classifier_proba = pd.DataFrame(
    np.hstack(
        (np.arange(0, 31, 1).reshape(-1, 1), classifier.predict_proba(classifier_data.values))
    ),
    columns=["time", "non_working_proba", "working_proba"],
)
non_working_proba = classifier_proba[["time", "non_working_proba"]].rename(
    columns={"non_working_proba": "proba"}
)
working_proba = classifier_proba[["time", "working_proba"]].rename(
    columns={"working_proba": "proba"}
)

In [ ]:
from cpu.utils.plot_recorders import plot_recorders

plot_recorders(
    {
        "diag": diag,
        "working_diag_proba": working_proba,
        "non_working_diag_proba": non_working_proba,
    },
    [[("time", "fan_working_diagnostic"), ("time", "proba")]],
    width=800,
    height=600,
    legend={"x": 0.75, "y": 0.25, "font": {"size": 20}, "orientation": "h", "xanchor": "center"},
)

In [ ]:
diag.to_csv(f"data/fan_working_diagnostic.csv", index=False)